# Matching the games with the team stats



In [2]:
import pandas as pd
import os
import numpy as np

In [ ]:
rankings=pd.read_csv('/content/ranking.csv')
teams=pd.read_csv('/content/teams.csv')
games=pd.read_csv('/content/games.csv')
details=pd.read_csv('/content/games_details.csv',low_memory=False)

Turns Road and Home records into percentages

In [ ]:
def calculate_home_win_percentage(row):
  results=row.HOME_RECORD
  results=results.split('-')
  wins=int(results[0]) 
  losses=int(results[1])
  total=wins+losses
  final=0
  if total > 0 :
    final=wins/total 
  return final

def calculate_away_win_percentage(row):
  results=row.ROAD_RECORD
  results=results.split('-')
  wins=int(results[0]) 
  losses=int(results[1])
  total=wins+losses
  final=0
  if total > 0 :
    final=wins/total 
  return final

Matches ball game with home and away teams status after the match

In [ ]:
def matchBallGameHomeDate(row,rankings) : 
  final='0-0'
  date=row.GAME_DATE_EST
  home_team=row.HOME_TEAM_ID
  home=rankings.loc[(rankings['TEAM_ID']==home_team) & (rankings['STANDINGSDATE'].str.startswith(date))]
  #final=home.HOME_RECORD.values[0]
  final=home.values[0]
  return final
 
def matchBallGameAwayDate(row,rankings) : 
  final='0-0'
  date=row.GAME_DATE_EST
  away_team=row.VISITOR_TEAM_ID
  away=rankings.loc[(rankings['TEAM_ID']==away_team) & (rankings['STANDINGSDATE'].str.startswith(date))]  
  final=away.values[0]
  return final

Matches team data with game data

In [ ]:
print(games)

      GAME_DATE_EST   GAME_ID GAME_STATUS_TEXT  HOME_TEAM_ID  VISITOR_TEAM_ID  \
0        2022-03-12  22101005            Final    1610612748       1610612750   
1        2022-03-12  22101006            Final    1610612741       1610612739   
2        2022-03-12  22101007            Final    1610612759       1610612754   
3        2022-03-12  22101008            Final    1610612744       1610612749   
4        2022-03-12  22101009            Final    1610612743       1610612761   
...             ...       ...              ...           ...              ...   
25791    2014-10-06  11400007            Final    1610612737       1610612740   
25792    2014-10-06  11400004            Final    1610612741       1610612764   
25793    2014-10-06  11400005            Final    1610612747       1610612743   
25794    2014-10-05  11400002            Final    1610612761       1610612758   
25795    2014-10-04  11400001            Final    1610612748       1610612740   

       SEASON  TEAM_ID_home

In [ ]:
games.drop_duplicates(subset=['GAME_DATE_EST','GAME_ID'],inplace=True)

In [ ]:
print(games)

      GAME_DATE_EST   GAME_ID GAME_STATUS_TEXT  HOME_TEAM_ID  VISITOR_TEAM_ID  \
0        2022-03-12  22101005            Final    1610612748       1610612750   
1        2022-03-12  22101006            Final    1610612741       1610612739   
2        2022-03-12  22101007            Final    1610612759       1610612754   
3        2022-03-12  22101008            Final    1610612744       1610612749   
4        2022-03-12  22101009            Final    1610612743       1610612761   
...             ...       ...              ...           ...              ...   
25791    2014-10-06  11400007            Final    1610612737       1610612740   
25792    2014-10-06  11400004            Final    1610612741       1610612764   
25793    2014-10-06  11400005            Final    1610612747       1610612743   
25794    2014-10-05  11400002            Final    1610612761       1610612758   
25795    2014-10-04  11400001            Final    1610612748       1610612740   

       SEASON  TEAM_ID_home

In [ ]:
full_games=games
full_games['HOME_TEAM_DATA'] = full_games.apply(lambda row: matchBallGameHomeDate(row,rankings), axis=1)
full_games['AWAY_TEAM_DATA'] = full_games.apply(lambda row: matchBallGameAwayDate(row,rankings), axis=1)
print(full_games)

      GAME_DATE_EST   GAME_ID GAME_STATUS_TEXT  HOME_TEAM_ID  VISITOR_TEAM_ID  \
0        2022-03-12  22101005            Final    1610612748       1610612750   
1        2022-03-12  22101006            Final    1610612741       1610612739   
2        2022-03-12  22101007            Final    1610612759       1610612754   
3        2022-03-12  22101008            Final    1610612744       1610612749   
4        2022-03-12  22101009            Final    1610612743       1610612761   
...             ...       ...              ...           ...              ...   
25791    2014-10-06  11400007            Final    1610612737       1610612740   
25792    2014-10-06  11400004            Final    1610612741       1610612764   
25793    2014-10-06  11400005            Final    1610612747       1610612743   
25794    2014-10-05  11400002            Final    1610612761       1610612758   
25795    2014-10-04  11400001            Final    1610612748       1610612740   

       SEASON  TEAM_ID_home

Categorize each team's data in columns

In [ ]:
full_testing=full_games.copy()

full_testing[['H_TEAM_ID','H_LEAGUE_ID','H_SEASON_ID','H_STANDINGSDATE','H_CONFERENCE','H_TEAM','H_G','H_W','H_L','H_W_PCT','H_HOME_RECORD',
              'H_ROAD_RECORD','H_RETURNTOPLAY']] = pd.DataFrame(full_testing.HOME_TEAM_DATA.tolist(), index= full_testing.index)
print(full_testing)
full_testing[['A_TEAM_ID','A_LEAGUE_ID','A_SEASON_ID','A_STANDINGSDATE','A_CONFERENCE','A_TEAM','A_G','A_W','A_L','A_W_PCT','A_HOME_RECORD',
              'A_ROAD_RECORD','A_RETURNTOPLAY']] = pd.DataFrame(full_testing.AWAY_TEAM_DATA.tolist(), index= full_testing.index)
print(full_testing)
full_testing.to_csv('game_data_teams_unfinished.csv')
final_full_games=full_testing.copy()
final_full_games.drop('HOME_TEAM_DATA', inplace=True, axis=1)
final_full_games.drop('AWAY_TEAM_DATA', inplace=True, axis=1)
final_full_games.to_csv('game_data_teams.csv')
print(final_full_games)
print(games)

      GAME_DATE_EST   GAME_ID GAME_STATUS_TEXT  HOME_TEAM_ID  VISITOR_TEAM_ID  \
0        2022-03-12  22101005            Final    1610612748       1610612750   
1        2022-03-12  22101006            Final    1610612741       1610612739   
2        2022-03-12  22101007            Final    1610612759       1610612754   
3        2022-03-12  22101008            Final    1610612744       1610612749   
4        2022-03-12  22101009            Final    1610612743       1610612761   
...             ...       ...              ...           ...              ...   
25791    2014-10-06  11400007            Final    1610612737       1610612740   
25792    2014-10-06  11400004            Final    1610612741       1610612764   
25793    2014-10-06  11400005            Final    1610612747       1610612743   
25794    2014-10-05  11400002            Final    1610612761       1610612758   
25795    2014-10-04  11400001            Final    1610612748       1610612740   

       SEASON  TEAM_ID_home

In [ ]:
games_teams=final_full_games.copy()

# Initializing Generic Functions


In [ ]:
import pandas as pd
import os
import numpy as np
rankings=pd.read_csv('/content/ranking.csv')
teams=pd.read_csv('/content/teams.csv')
games=pd.read_csv('/content/games.csv')
#details=pd.read_csv('/content/games_details.csv',low_memory=False)
games_teams=pd.read_csv('/content/game_data_teams.csv',low_memory=False)
games_teams.drop('Unnamed: 0', inplace=True, axis=1)

In [22]:
def calculateOrderBasedOnDate(table) :
  copy=table.copy()
  copy=copy.sort_values(by = ['GAME_DATE_EST'], ascending = True,axis=0)
  return copy.index

In [23]:
def createColumns(columns,mode):
  final=[]
  for column in columns :
    if mode==0 :
      home_season_string='H_S_'
      away_season_string='A_S_'
      home_season_allowed='_A'
      away_season_allowed='_A'
      home_string='_home'
      away_string='_away'
      table=[[],[],[],[]]
      table_allowed=[[],[],[],[]]
    else :
      home_season_string='H_L5_'
      away_season_string='A_L5_'
      home_season_allowed='_A'
      away_season_allowed='_A'
      home_string='_home'
      away_string='_away'
      table=[[],[],[],[],[],[],[],[]]
      table_allowed=[[],[],[],[],[],[],[],[]]
    home_season_string= home_season_string + column
    away_season_string= away_season_string + column
    home_season_allowed= home_season_string + home_season_allowed
    away_season_allowed= away_season_string + away_season_allowed
    home_string=column + home_string
    away_string=column + away_string
    if mode==0 :
      table1=[home_season_string,home_season_string,home_string]
      table2=[home_season_string,away_season_string,away_string]
      table3=[away_season_string,home_season_string,home_string]
      table4=[away_season_string,away_season_string,away_string]
      table[0]=table1
      table[1]=table2
      table[2]=table3
      table[3]=table4
      table1_allowed=[home_season_allowed,home_season_allowed,away_string]
      table2_allowed=[home_season_allowed,away_season_allowed,home_string]
      table3_allowed=[away_season_allowed,home_season_allowed,away_string]
      table4_allowed=[away_season_allowed,away_season_allowed,home_string]
      table_allowed[0]=table1_allowed
      table_allowed[1]=table2_allowed
      table_allowed[2]=table3_allowed
      table_allowed[3]=table4_allowed
    else :
      table1=[home_season_string,home_season_string,home_string,home_string]
      table2=[home_season_string,home_season_string,home_string,away_string]
      table3=[home_season_string,away_season_string,away_string,home_string]
      table4=[home_season_string,away_season_string,away_string,away_string]
      table5=[away_season_string,home_season_string,home_string,home_string]
      table6=[away_season_string,home_season_string,home_string,away_string]
      table7=[away_season_string,away_season_string,away_string,home_string]
      table8=[away_season_string,away_season_string,away_string,away_string]
      table[0]=table1
      table[1]=table2
      table[2]=table3
      table[3]=table4
      table[4]=table5
      table[5]=table6
      table[6]=table7
      table[7]=table8
      table1_allowed=[home_season_allowed,home_season_allowed,away_string,away_string]
      table2_allowed=[home_season_allowed,home_season_allowed,away_string,home_string]
      table3_allowed=[home_season_allowed,away_season_allowed,home_string,away_string]
      table4_allowed=[home_season_allowed,away_season_allowed,home_string,home_string]
      table5_allowed=[away_season_allowed,home_season_allowed,away_string,away_string]
      table6_allowed=[away_season_allowed,home_season_allowed,away_string,home_string]
      table7_allowed=[away_season_allowed,away_season_allowed,home_string,away_string]
      table8_allowed=[away_season_allowed,away_season_allowed,home_string,home_string]
      table_allowed[0]=table1_allowed
      table_allowed[1]=table2_allowed
      table_allowed[2]=table3_allowed
      table_allowed[3]=table4_allowed
      table_allowed[4]=table5_allowed
      table_allowed[5]=table6_allowed
      table_allowed[6]=table7_allowed
      table_allowed[7]=table8_allowed
    final.append(table)
    final.append(table_allowed)
  return final

In [24]:
def genericFunctionCaller(games_teams,columns,function_called): 
  max_games=82
  #65
  ids_of_teams=[1610612737,1610612738,1610612740,1610612741,1610612742,1610612743,1610612745,
                1610612746,1610612747,1610612748,1610612749,1610612750,1610612751,1610612752,
                1610612753,1610612754,1610612755,1610612756,1610612757,1610612758,1610612759,
                1610612760,1610612761,1610612762,1610612763,1610612764,1610612765,1610612766,1610612739,1610612744]
  number_of_2019_games=[67,72,72,65,75,73,72,72,71,73,73,64,72,66,73,73,73,73,74,72,71,72,72,72,73,72,66,65,65,65]
  for k in range(15):
    for i in range(2003,2021,1) :
      if i==2011:
        games_teams=function_called(games_teams,i,ids_of_teams[k],66,columns)
      elif i==2019:
        games_teams=function_called(games_teams,i,ids_of_teams[k],number_of_2019_games[k],columns)
      elif i==2020:
        games_teams=function_called(games_teams,i,ids_of_teams[k],72,columns)
      elif i==2021:
        games_teams=function_called(games_teams,i,ids_of_teams[k],68,columns)
      elif i==2012 :
        if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
          games_teams=function_called(games_teams,i,ids_of_teams[k],81,columns)
        else :
          games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
      else :
        games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)

  for k in range(15,27,1):
    for i in range(2003,2021,1) :
      if i==2011:
        games_teams=function_called(games_teams,i,ids_of_teams[k],66,columns)
      elif i==2019:
        games_teams=function_called(games_teams,i,ids_of_teams[k],number_of_2019_games[k],columns)
      elif i==2020:
        games_teams=function_called(games_teams,i,ids_of_teams[k],72,columns)
      elif i==2021:
        games_teams=function_called(games_teams,i,ids_of_teams[k],68,columns)
      elif i==2012 :
        if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
          games_teams=function_called(games_teams,i,ids_of_teams[k],81,columns)
        else :
          games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
      else :
        games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
  for k in range(27,28,1):
    for i in range(2004,2021,1) :
      if i==2011:
        games_teams=function_called(games_teams,i,ids_of_teams[k],66,columns)
      elif i==2019:
        games_teams=function_called(games_teams,i,ids_of_teams[k],number_of_2019_games[k],columns)
      elif i==2020:
        games_teams=function_called(games_teams,i,ids_of_teams[k],72,columns)
      elif i==2021:
        games_teams=function_called(games_teams,i,ids_of_teams[k],68,columns)
      elif i==2012 :
        if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
          games_teams=function_called(games_teams,i,ids_of_teams[k],81,columns)
        else :
          games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
      else :
        games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
  for k in range(28,30,1):
    for i in range(2003,2021,1) :
      if i==2011:
        games_teams=function_called(games_teams,i,ids_of_teams[k],66,columns)
      elif i==2019:
        games_teams=function_called(games_teams,i,ids_of_teams[k],number_of_2019_games[k],columns)
      elif i==2020:
        games_teams=function_called(games_teams,i,ids_of_teams[k],72,columns)
      elif i==2021:
        games_teams=function_called(games_teams,i,ids_of_teams[k],68,columns)
      elif i==2012 :
        if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
          games_teams=function_called(games_teams,i,ids_of_teams[k],81,columns)
        else :
          games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
      else :
        games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
  return games_teams

In [25]:
def genericFunctionCallerSplitFirst(games_teams,columns,function_called): 
  max_games=82
  #65
  ids_of_teams=[1610612737,1610612738,1610612740,1610612741,1610612742,1610612743,1610612745,
                1610612746,1610612747,1610612748,1610612749,1610612750,1610612751,1610612752,
                1610612753,1610612754,1610612755,1610612756,1610612757,1610612758,1610612759,
                1610612760,1610612761,1610612762,1610612763,1610612764,1610612765,1610612766,1610612739,1610612744]
  number_of_2019_games=[67,72,72,65,75,73,72,72,71,73,73,64,72,66,73,73,73,73,74,72,71,72,72,72,73,72,66,65,65,65]
  for k in range(15):
    for i in range(2003,2021,1) :
      if i==2011:
        games_teams=function_called(games_teams,i,ids_of_teams[k],66,columns)
      elif i==2019:
        games_teams=function_called(games_teams,i,ids_of_teams[k],number_of_2019_games[k],columns)
      elif i==2020:
        games_teams=function_called(games_teams,i,ids_of_teams[k],72,columns)
      elif i==2021:
        games_teams=function_called(games_teams,i,ids_of_teams[k],68,columns)
      elif i==2012 :
        if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
          games_teams=function_called(games_teams,i,ids_of_teams[k],81,columns)
        else :
          games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
      else :
        games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
  return games_teams

In [26]:
def genericFunctionCallerSplitSecond(games_teams,columns,function_called): 
  max_games=82
  #65
  ids_of_teams=[1610612737,1610612738,1610612740,1610612741,1610612742,
                1610612743,1610612745,1610612746,1610612747,1610612748,1610612749,
                1610612750,1610612751,1610612752,1610612753,1610612754,1610612755,
                1610612756,1610612757,1610612758,1610612759,1610612760,1610612761,1610612762,1610612763,1610612764,1610612765,1610612766,1610612739,1610612744]
  number_of_2019_games=[67,72,72,65,75,73,72,72,71,73,73,64,72,66,73,73,73,73,74,72,71,72,72,72,73,72,66,65,65,65]
  for k in range(15,27,1):
    for i in range(2003,2021,1) :
      if i==2011:
        games_teams=function_called(games_teams,i,ids_of_teams[k],66,columns)
      elif i==2019:
        games_teams=function_called(games_teams,i,ids_of_teams[k],number_of_2019_games[k],columns)
      elif i==2020:
        games_teams=function_called(games_teams,i,ids_of_teams[k],72,columns)
      elif i==2021:
        games_teams=function_called(games_teams,i,ids_of_teams[k],68,columns)
      elif i==2012 :
        if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
          games_teams=function_called(games_teams,i,ids_of_teams[k],81,columns)
        else :
          games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
      else :
        games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
  for k in range(27,28,1):
    for i in range(2004,2021,1) :
      if i==2011:
        games_teams=function_called(games_teams,i,ids_of_teams[k],66,columns)
      elif i==2019:
        games_teams=function_called(games_teams,i,ids_of_teams[k],number_of_2019_games[k],columns)
      elif i==2020:
        games_teams=function_called(games_teams,i,ids_of_teams[k],72,columns)
      elif i==2021:
        games_teams=function_called(games_teams,i,ids_of_teams[k],68,columns)
      elif i==2012 :
        if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
          games_teams=function_called(games_teams,i,ids_of_teams[k],81,columns)
        else :
          games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
      else :
        games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
  for k in range(28,30,1):
    for i in range(2003,2021,1) :
      if i==2011:
        games_teams=function_called(games_teams,i,ids_of_teams[k],66,columns)
      elif i==2019:
        games_teams=function_called(games_teams,i,ids_of_teams[k],number_of_2019_games[k],columns)
      elif i==2020:
        games_teams=function_called(games_teams,i,ids_of_teams[k],72,columns)
      elif i==2021:
        games_teams=function_called(games_teams,i,ids_of_teams[k],68,columns)
      elif i==2012 :
        if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
          games_teams=function_called(games_teams,i,ids_of_teams[k],81,columns)
        else :
          games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
      else :
        games_teams=function_called(games_teams,i,ids_of_teams[k],max_games,columns)
  return games_teams

# Calculating Season Averages, Calculating L5 Game Averages for the basic statistics, defining their functions

In [ ]:
columns=['PTS','FG_PCT','FT_PCT','FG3_PCT','AST','REB']

In [ ]:
games_teams['H_S_PTS']=0
games_teams['H_S_FG_PCT']=0
games_teams['H_S_FT_PCT']=0
games_teams['H_S_FG3_PCT']=0
games_teams['H_S_AST']=0
games_teams['H_S_REB']=0
games_teams['H_S_PTS_A']=0
games_teams['H_S_FG_PCT_A']=0
games_teams['H_S_FT_PCT_A']=0
games_teams['H_S_FG3_PCT_A']=0
games_teams['H_S_AST_A']=0
games_teams['H_S_REB_A']=0
games_teams['A_S_PTS']=0
games_teams['A_S_FG_PCT']=0
games_teams['A_S_FT_PCT']=0
games_teams['A_S_FG3_PCT']=0
games_teams['A_S_AST']=0
games_teams['A_S_REB']=0
games_teams['A_S_PTS_A']=0
games_teams['A_S_FG_PCT_A']=0
games_teams['A_S_FT_PCT_A']=0
games_teams['A_S_FG3_PCT_A']=0
games_teams['A_S_AST_A']=0
games_teams['A_S_REB_A']=0

Initializing The Season Functions


In [ ]:
def calculateTheSeasonAveragesGeneric(table,year,team,max_games,column_names):
  columns=createColumns(column_names,0)
  results=table.loc[((table['HOME_TEAM_ID']==team) | (table['VISITOR_TEAM_ID']==team)) & (table['SEASON']==year)]
  year_classify=20000 + year  
  results=results.loc[(results['H_SEASON_ID']==year_classify)|(results['A_SEASON_ID']==year_classify)]
  games_played=2
  while games_played < max_games :
    game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
    previous_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-1)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-1)))]
    games_played=games_played+1 
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
        for column in columns :
          table.loc[game_info.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0] 
          results.loc[game_info.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0]     
      else :
        for column in columns :
          table.loc[game_info.index,[column[1][0]]]=previous_game_info[column[1][1]].values[0] + previous_game_info[column[1][2]].values[0] 
          results.loc[game_info.index,[column[1][0]]]=previous_game_info[column[1][1]].values[0] + previous_game_info[column[1][2]].values[0]
    else :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
        for column in columns :
          table.loc[game_info.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0] 
          results.loc[game_info.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0]
      else :
        for column in columns :
          table.loc[game_info.index,[column[3][0]]]=previous_game_info[column[3][1]].values[0] + previous_game_info[column[3][2]].values[0] 
          results.loc[game_info.index,[column[3][0]]]=previous_game_info[column[3][1]].values[0] + previous_game_info[column[3][2]].values[0]
  #
  game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
  previous_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-1)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-1)))]
  game_info1=game_info.loc[game_info['HOME_TEAM_ID']==team]
  game_info2=game_info.loc[game_info['VISITOR_TEAM_ID']==team]
  if (not(game_info1.empty)) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
        for column in columns :
          table.loc[game_info1.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0] 
          results.loc[game_info1.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0]    
      else :
        for column in columns :
          table.loc[game_info1.index,[column[1][0]]]=previous_game_info[column[1][1]].values[0] + previous_game_info[column[1][2]].values[0] 
          results.loc[game_info1.index,[column[1][0]]]=previous_game_info[column[1][1]].values[0] + previous_game_info[column[1][2]].values[0]
  if (not(game_info2.empty)) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
        for column in columns :
          table.loc[game_info2.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0] 
          results.loc[game_info2.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0]
      else :
        for column in columns :
          table.loc[game_info2.index,[column[3][0]]]=previous_game_info[column[3][1]].values[0] + previous_game_info[column[3][2]].values[0] 
          results.loc[game_info2.index,[column[3][0]]]=previous_game_info[column[3][1]].values[0] + previous_game_info[column[3][2]].values[0]
  return table

In [ ]:
def fixPostSeasonAveragesGeneric(table,year,team,max_games,column_names):
  columns=createColumns(column_names,0)
  results=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_G']==max_games)) | ((table['VISITOR_TEAM_ID']==team)&(table['A_G']==max_games))) & (table['SEASON']==year)]
  last_regular_season_game_date=results['GAME_DATE_EST'].min()
  lrsg=results.loc[results['GAME_DATE_EST'].str.startswith(last_regular_season_game_date)]
  results2=results.copy()
  results2=results2.drop(index=lrsg.index)
  playoff_games=results.loc[results2.index]
  previous_game_info=lrsg
  game_info1=playoff_games.loc[playoff_games['HOME_TEAM_ID']==team]
  game_info2=playoff_games.loc[playoff_games['VISITOR_TEAM_ID']==team]
  if (not(game_info1.empty)) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
        for column in columns :
          table.loc[game_info1.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0]   
      else :
        for column in columns :
          table.loc[game_info1.index,[column[1][0]]]=previous_game_info[column[1][1]].values[0] + previous_game_info[column[1][2]].values[0] 
  if (not(game_info2.empty)) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
        for column in columns :
          table.loc[game_info2.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0] 
      else :
        for column in columns :
          table.loc[game_info2.index,[column[3][0]]]=previous_game_info[column[3][1]].values[0] + previous_game_info[column[3][2]].values[0] 
  
  return table 

In [ ]:
def calculateTheSeasonAveragesPerGameGeneric(table,year,team,max_games,column_names):
  columns=createColumns(column_names,0)
  results=table.loc[((table['HOME_TEAM_ID']==team) | (table['VISITOR_TEAM_ID']==team)) & (table['SEASON']==year)]
  year_classify=20000 + year  
  results=results.loc[(results['H_SEASON_ID']==year_classify)|(results['A_SEASON_ID']==year_classify)]
  games_played=2
  while games_played < max_games :
    game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
        for column in columns :
          table.loc[game_info.index,[column[0][0]]]=game_info[column[0][0]].values[0]/(games_played-1)     
    else :
        for column in columns :
          table.loc[game_info.index,[column[2][0]]]=game_info[column[2][0]].values[0]/(games_played-1)    
    games_played=games_played + 1
  games_max=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
  last_regular_season_game_date=games_max['GAME_DATE_EST'].min()
  lrsg=games_max.loc[games_max['GAME_DATE_EST'].str.startswith(last_regular_season_game_date)]
  if (lrsg['HOME_TEAM_ID'].values[0]==team) :
        for column in columns :
          table.loc[lrsg.index,[column[0][0]]]=lrsg[column[0][0]].values[0]/(games_played-1)     
  else :
        for column in columns :
          table.loc[lrsg.index,[column[2][0]]]=lrsg[column[2][0]].values[0]/(games_played-1) 
  results2=games_max.copy()
  results2=results2.drop(index=lrsg.index)
  playoff_games=games_max.loc[results2.index]
  game_info1=playoff_games.loc[playoff_games['HOME_TEAM_ID']==team]
  game_info2=playoff_games.loc[playoff_games['VISITOR_TEAM_ID']==team]
  if (not(game_info1.empty)) :
      for column in columns :
          table.loc[game_info1.index,[column[0][0]]]=game_info1[column[0][0]].values[0]/(max_games)            
  if (not(game_info2.empty)) :
      for column in columns :
          table.loc[game_info2.index,[column[2][0]]]=game_info2[column[2][0]].values[0]/(max_games)   
  return table 

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesGeneric)

In [ ]:
games_teams.to_csv('games_ptstoreb_total.csv')

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,fixPostSeasonAveragesGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,fixPostSeasonAveragesGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesPerGameGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesPerGameGeneric)

In [ ]:
games_teams.to_csv('games_ptstoreb_averages.csv')

In [ ]:
import pandas as pd
import os
games_teams=pd.read_csv('/content/games_ptstoreb_averages.csv')
games_teams.drop('Unnamed: 0', inplace=True, axis=1)

Initializing the L5 game functions

In [ ]:
def calculateTheSeasonAveragesGenericL5(table,year,team,max_games,column_names):
  columns=createColumns(column_names,1)
  results=table.loc[((table['HOME_TEAM_ID']==team) | (table['VISITOR_TEAM_ID']==team)) & (table['SEASON']==year)]
  year_classify=20000 + year  
  results=results.loc[(results['H_SEASON_ID']==year_classify)|(results['A_SEASON_ID']==year_classify)]
  
  games_played=2
  while games_played < 7 :
    game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
    previous_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-1)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-1)))]
    games_played=games_played+1 
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          for column in columns :
            table.loc[game_info.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0] 
            results.loc[game_info.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0]  
      else :
          for column in columns :
            table.loc[game_info.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0] 
            results.loc[game_info.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0] 
    else :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          for column in columns :
            table.loc[game_info.index,[column[4][0]]]=previous_game_info[column[4][1]].values[0] + previous_game_info[column[4][2]].values[0] 
            results.loc[game_info.index,[column[4][0]]]=previous_game_info[column[4][1]].values[0] + previous_game_info[column[4][2]].values[0]  
      
      else :
          for column in columns :
            table.loc[game_info.index,[column[6][0]]]=previous_game_info[column[6][1]].values[0] + previous_game_info[column[6][2]].values[0] 
            results.loc[game_info.index,[column[6][0]]]=previous_game_info[column[6][1]].values[0] + previous_game_info[column[6][2]].values[0] 
       
  
  while games_played < max_games :
    game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
    previous_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-1)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-1)))]
    previous6_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-6)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-6)))]
    games_played=games_played+1 
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0] - previous6_game_info[column[0][3]].values[0] 
              results.loc[game_info.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0] - previous6_game_info[column[0][3]].values[0] 
          else :
            for column in columns :
              table.loc[game_info.index,[column[1][0]]]=previous_game_info[column[1][1]].values[0] + previous_game_info[column[1][2]].values[0] - previous6_game_info[column[1][3]].values[0] 
              results.loc[game_info.index,[column[1][0]]]=previous_game_info[column[1][1]].values[0] + previous_game_info[column[1][2]].values[0] - previous6_game_info[column[1][3]].values[0] 
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0] - previous6_game_info[column[2][3]].values[0] 
              results.loc[game_info.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0] - previous6_game_info[column[2][3]].values[0] 
        else :
            for column in columns :
              table.loc[game_info.index,[column[3][0]]]=previous_game_info[column[3][1]].values[0] + previous_game_info[column[3][2]].values[0] - previous6_game_info[column[3][3]].values[0] 
              results.loc[game_info.index,[column[3][0]]]=previous_game_info[column[3][1]].values[0] + previous_game_info[column[3][2]].values[0] - previous6_game_info[column[3][3]].values[0] 
    else :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info.index,[column[4][0]]]=previous_game_info[column[4][1]].values[0] + previous_game_info[column[4][2]].values[0] - previous6_game_info[column[4][3]].values[0] 
              results.loc[game_info.index,[column[4][0]]]=previous_game_info[column[4][1]].values[0] + previous_game_info[column[4][2]].values[0] - previous6_game_info[column[4][3]].values[0] 
          else :
            for column in columns :
              table.loc[game_info.index,[column[5][0]]]=previous_game_info[column[5][1]].values[0] + previous_game_info[column[5][2]].values[0] - previous6_game_info[column[5][3]].values[0] 
              results.loc[game_info.index,[column[5][0]]]=previous_game_info[column[5][1]].values[0] + previous_game_info[column[5][2]].values[0] - previous6_game_info[column[5][3]].values[0] 
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info.index,[column[6][0]]]=previous_game_info[column[6][1]].values[0] + previous_game_info[column[6][2]].values[0] - previous6_game_info[column[6][3]].values[0] 
              results.loc[game_info.index,[column[6][0]]]=previous_game_info[column[6][1]].values[0] + previous_game_info[column[6][2]].values[0] - previous6_game_info[column[6][3]].values[0] 
        else :
            for column in columns :
              table.loc[game_info.index,[column[7][0]]]=previous_game_info[column[7][1]].values[0] + previous_game_info[column[7][2]].values[0] - previous6_game_info[column[7][3]].values[0] 
              results.loc[game_info.index,[column[7][0]]]=previous_game_info[column[7][1]].values[0] + previous_game_info[column[7][2]].values[0] - previous6_game_info[column[7][3]].values[0] 
  #
  game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
  previous_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-1)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-1)))]
  previous6_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-6)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-6)))]
  game_info1=game_info.loc[game_info['HOME_TEAM_ID']==team]
  game_info2=game_info.loc[game_info['VISITOR_TEAM_ID']==team]
  if (not(game_info1.empty)) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info1.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0] - previous6_game_info[column[0][3]].values[0] 
              results.loc[game_info1.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0] - previous6_game_info[column[0][3]].values[0] 
          else :
            for column in columns :
              table.loc[game_info1.index,[column[1][0]]]=previous_game_info[column[1][1]].values[0] + previous_game_info[column[1][2]].values[0] - previous6_game_info[column[1][3]].values[0] 
              results.loc[game_info1.index,[column[1][0]]]=previous_game_info[column[1][1]].values[0] + previous_game_info[column[1][2]].values[0] - previous6_game_info[column[1][3]].values[0] 
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info1.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0] - previous6_game_info[column[2][3]].values[0] 
              results.loc[game_info1.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0] - previous6_game_info[column[2][3]].values[0] 
        else :
            for column in columns :
              table.loc[game_info1.index,[column[3][0]]]=previous_game_info[column[3][1]].values[0] + previous_game_info[column[3][2]].values[0] - previous6_game_info[column[3][3]].values[0] 
              results.loc[game_info1.index,[column[3][0]]]=previous_game_info[column[3][1]].values[0] + previous_game_info[column[3][2]].values[0] - previous6_game_info[column[3][3]].values[0] 
  if (not(game_info2.empty)) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info2.index,[column[4][0]]]=previous_game_info[column[4][1]].values[0] + previous_game_info[column[4][2]].values[0] - previous6_game_info[column[4][3]].values[0] 
              results.loc[game_info2.index,[column[4][0]]]=previous_game_info[column[4][1]].values[0] + previous_game_info[column[4][2]].values[0] - previous6_game_info[column[4][3]].values[0] 
          else :
            for column in columns :
              table.loc[game_info2.index,[column[5][0]]]=previous_game_info[column[5][1]].values[0] + previous_game_info[column[5][2]].values[0] - previous6_game_info[column[5][3]].values[0] 
              results.loc[game_info2.index,[column[5][0]]]=previous_game_info[column[5][1]].values[0] + previous_game_info[column[5][2]].values[0] - previous6_game_info[column[5][3]].values[0] 
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info2.index,[column[6][0]]]=previous_game_info[column[6][1]].values[0] + previous_game_info[column[6][2]].values[0] - previous6_game_info[column[6][3]].values[0] 
              results.loc[game_info2.index,[column[6][0]]]=previous_game_info[column[6][1]].values[0] + previous_game_info[column[6][2]].values[0] - previous6_game_info[column[6][3]].values[0] 
        else :
            for column in columns :
              table.loc[game_info2.index,[column[7][0]]]=previous_game_info[column[7][1]].values[0] + previous_game_info[column[7][2]].values[0] - previous6_game_info[column[7][3]].values[0] 
              results.loc[game_info2.index,[column[7][0]]]=previous_game_info[column[7][1]].values[0] + previous_game_info[column[7][2]].values[0] - previous6_game_info[column[7][3]].values[0]

  return table 

In [ ]:
def fixPostSeasonAveragesGenericL5(table,year,team,max_games,column_names) :
  columns=createColumns(column_names,1)
  results=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_G']==max_games)) | ((table['VISITOR_TEAM_ID']==team)&(table['A_G']==max_games))) & (table['SEASON']==year)]
  results_regular=table.loc[((table['HOME_TEAM_ID']==team) | (table['VISITOR_TEAM_ID']==team)) & (table['SEASON']==year)]
  last_regular_season_game_date=results['GAME_DATE_EST'].min()
  lrsg=results.loc[results['GAME_DATE_EST'].str.startswith(last_regular_season_game_date)]
  last=lrsg.copy()
  results2=results.copy()
  results=results.drop(index=lrsg.index)
  amount_of_games=len(results)
  index=calculateOrderBasedOnDate(results)
  games_back=5
  amount_fixed=0
  while amount_fixed < amount_of_games :
    game_info_temp=results.loc[index[amount_fixed]]
    game_info=results.loc[results['GAME_ID']==game_info_temp['GAME_ID']]
    if amount_fixed==0 :
      previous_game_info=last
    else :
      previous_game_info_temp=results.loc[index[amount_fixed-1]]
      previous_game_info=results.loc[results['GAME_ID']==previous_game_info_temp['GAME_ID']]
    if (5-amount_fixed)>0 :
      previous6_game_info=results_regular.loc[((results_regular['HOME_TEAM_ID']==team)&(results_regular['H_G']==(max_games-(5-amount_fixed))))|((results_regular['VISITOR_TEAM_ID']==team)&(results_regular['A_G']==(max_games-(5-amount_fixed))))]
    elif (5-amount_fixed)==0 :
      previous6_game_info=last
    else :
      previous6_game_info_temp=results.loc[index[amount_fixed-6]]
      previous6_game_info=results.loc[results['GAME_ID']==previous6_game_info_temp['GAME_ID']]
    amount_fixed=amount_fixed+1
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0] - previous6_game_info[column[0][3]].values[0] 
              results.loc[game_info.index,[column[0][0]]]=previous_game_info[column[0][1]].values[0] + previous_game_info[column[0][2]].values[0] - previous6_game_info[column[0][3]].values[0] 
          else :
            for column in columns :
              table.loc[game_info.index,[column[1][0]]]=previous_game_info[column[1][1]].values[0] + previous_game_info[column[1][2]].values[0] - previous6_game_info[column[1][3]].values[0] 
              results.loc[game_info.index,[column[1][0]]]=previous_game_info[column[1][1]].values[0] + previous_game_info[column[1][2]].values[0] - previous6_game_info[column[1][3]].values[0] 
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0] - previous6_game_info[column[2][3]].values[0] 
              results.loc[game_info.index,[column[2][0]]]=previous_game_info[column[2][1]].values[0] + previous_game_info[column[2][2]].values[0] - previous6_game_info[column[2][3]].values[0] 
        else :
            for column in columns :
              table.loc[game_info.index,[column[3][0]]]=previous_game_info[column[3][1]].values[0] + previous_game_info[column[3][2]].values[0] - previous6_game_info[column[3][3]].values[0] 
              results.loc[game_info.index,[column[3][0]]]=previous_game_info[column[3][1]].values[0] + previous_game_info[column[3][2]].values[0] - previous6_game_info[column[3][3]].values[0] 
    else :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info.index,[column[4][0]]]=previous_game_info[column[4][1]].values[0] + previous_game_info[column[4][2]].values[0] - previous6_game_info[column[4][3]].values[0] 
              results.loc[game_info.index,[column[4][0]]]=previous_game_info[column[4][1]].values[0] + previous_game_info[column[4][2]].values[0] - previous6_game_info[column[4][3]].values[0] 
          else :
            for column in columns :
              table.loc[game_info.index,[column[5][0]]]=previous_game_info[column[5][1]].values[0] + previous_game_info[column[5][2]].values[0] - previous6_game_info[column[5][3]].values[0] 
              results.loc[game_info.index,[column[5][0]]]=previous_game_info[column[5][1]].values[0] + previous_game_info[column[5][2]].values[0] - previous6_game_info[column[5][3]].values[0] 
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            for column in columns :
              table.loc[game_info.index,[column[6][0]]]=previous_game_info[column[6][1]].values[0] + previous_game_info[column[6][2]].values[0] - previous6_game_info[column[6][3]].values[0] 
              results.loc[game_info.index,[column[6][0]]]=previous_game_info[column[6][1]].values[0] + previous_game_info[column[6][2]].values[0] - previous6_game_info[column[6][3]].values[0] 
        else :
            for column in columns :
              table.loc[game_info.index,[column[7][0]]]=previous_game_info[column[7][1]].values[0] + previous_game_info[column[7][2]].values[0] - previous6_game_info[column[7][3]].values[0] 
              results.loc[game_info.index,[column[7][0]]]=previous_game_info[column[7][1]].values[0] + previous_game_info[column[7][2]].values[0] - previous6_game_info[column[7][3]].values[0] 
  return table

In [ ]:
def calculateTheSeasonAveragesPerGameGenericL5(table,year,team,max_games,column_names):
  columns=createColumns(column_names,1)
  results=table.loc[((table['HOME_TEAM_ID']==team) | (table['VISITOR_TEAM_ID']==team)) & (table['SEASON']==year)]
  year_classify=20000 + year  
  results=results.loc[(results['H_SEASON_ID']==year_classify)|(results['A_SEASON_ID']==year_classify)]
  games_played=2
  while games_played < 7 :
    game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
        for column in columns :
          table.loc[game_info.index,[column[0][0]]]=game_info[column[0][0]].values[0]/(games_played-1)     
    else :
        for column in columns :
          table.loc[game_info.index,[column[4][0]]]=game_info[column[4][0]].values[0]/(games_played-1)    
    games_played=games_played + 1
  while games_played < max_games :
    game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
        for column in columns :
          table.loc[game_info.index,[column[0][0]]]=game_info[column[0][0]].values[0]/5   
    else :
        for column in columns :
          table.loc[game_info.index,[column[4][0]]]=game_info[column[4][0]].values[0]/5   
    games_played=games_played + 1
  games_max=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
  last_regular_season_game_date=games_max['GAME_DATE_EST'].min()
  lrsg=games_max.loc[games_max['GAME_DATE_EST'].str.startswith(last_regular_season_game_date)]
  if (lrsg['HOME_TEAM_ID'].values[0]==team) :
        for column in columns :
          table.loc[lrsg.index,[column[0][0]]]=lrsg[column[0][0]].values[0]/5    
  else :
        for column in columns :
          table.loc[lrsg.index,[column[4][0]]]=lrsg[column[4][0]].values[0]/5
  results2=games_max.copy()
  results2=results2.drop(index=lrsg.index)
  playoff_games=games_max.loc[results2.index]
  amount_of_games=len(playoff_games)
  index=calculateOrderBasedOnDate(playoff_games)
  amount_fixed=0
  while amount_fixed < amount_of_games :
    game_info_temp=playoff_games.loc[index[amount_fixed]]
    game_info=playoff_games.loc[playoff_games['GAME_ID']==game_info_temp['GAME_ID']]
    amount_fixed=amount_fixed + 1
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
      for column in columns :
          table.loc[game_info.index,[column[0][0]]]=game_info[column[0][0]].values[0]/5          
    else :
      for column in columns :
          table.loc[game_info.index,[column[4][0]]]=game_info[column[4][0]].values[0]/5  
  return table 

In [ ]:
games_teams['H_L5_PTS']=0
games_teams['H_L5_FG_PCT']=0
games_teams['H_L5_FT_PCT']=0
games_teams['H_L5_FG3_PCT']=0
games_teams['H_L5_AST']=0
games_teams['H_L5_REB']=0
games_teams['H_L5_PTS_A']=0
games_teams['H_L5_FG_PCT_A']=0
games_teams['H_L5_FT_PCT_A']=0
games_teams['H_L5_FG3_PCT_A']=0
games_teams['H_L5_AST_A']=0
games_teams['H_L5_REB_A']=0
games_teams['A_L5_PTS']=0
games_teams['A_L5_FG_PCT']=0
games_teams['A_L5_FT_PCT']=0
games_teams['A_L5_FG3_PCT']=0
games_teams['A_L5_AST']=0
games_teams['A_L5_REB']=0
games_teams['A_L5_PTS_A']=0
games_teams['A_L5_FG_PCT_A']=0
games_teams['A_L5_FT_PCT_A']=0
games_teams['A_L5_FG3_PCT_A']=0
games_teams['A_L5_AST_A']=0
games_teams['A_L5_REB_A']=0

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesGenericL5)

In [ ]:
games_teams.to_csv('games_ptstoreb_averages_l5_total.csv')

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,fixPostSeasonAveragesGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,fixPostSeasonAveragesGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesPerGameGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesPerGameGenericL5)

In [ ]:
games_teams.to_csv('games_ptstoreb_averages_l5_averages.csv')

# Add the rest of the statistics, their averages, and win loss functions plus netrating calculation

In [ ]:
games_teams=pd.read_csv('/content/games_ptstoreb_averages_l5_averages.csv')
games_teams.drop('Unnamed: 0', inplace=True, axis=1)
details=pd.read_csv('/content/games_details.csv',low_memory=False)

In [ ]:
def addStatsToGames(row,details,team,column) :
  game=row['GAME_ID']
  team=row[team]
  data=details.loc[(details['GAME_ID']==game) & (details['TEAM_ID']==team)]
  column = data[column].fillna(0)
  sum=int(column.sum())
  return sum

In [ ]:
columns=[['STL','STL_home','STL_away'],['BLK','BLK_home','BLK_away'],['TO','TO_home','TO_away']]
for column in columns :
  games_teams[column[1]]=games_teams.apply(lambda row:addStatsToGames(row,details,'HOME_TEAM_ID',column[0]), axis=1)
for column in columns :
  games_teams[column[2]]=games_teams.apply(lambda row:addStatsToGames(row,details,'VISITOR_TEAM_ID',column[0]), axis=1)
print(games_teams)

      GAME_DATE_EST   GAME_ID GAME_STATUS_TEXT  HOME_TEAM_ID  VISITOR_TEAM_ID  \
0        2022-03-12  22101005            Final    1610612748       1610612750   
1        2022-03-12  22101006            Final    1610612741       1610612739   
2        2022-03-12  22101007            Final    1610612759       1610612754   
3        2022-03-12  22101008            Final    1610612744       1610612749   
4        2022-03-12  22101009            Final    1610612743       1610612761   
...             ...       ...              ...           ...              ...   
25762    2014-10-06  11400007            Final    1610612737       1610612740   
25763    2014-10-06  11400004            Final    1610612741       1610612764   
25764    2014-10-06  11400005            Final    1610612747       1610612743   
25765    2014-10-05  11400002            Final    1610612761       1610612758   
25766    2014-10-04  11400001            Final    1610612748       1610612740   

       SEASON  TEAM_ID_home

In [ ]:
columns=['STL','BLK','TO']

In [ ]:
games_teams['H_S_STL']=0
games_teams['H_S_BLK']=0
games_teams['H_S_TO']=0
games_teams['H_S_STL_A']=0
games_teams['H_S_BLK_A']=0
games_teams['H_S_TO_A']=0
games_teams['A_S_STL']=0
games_teams['A_S_BLK']=0
games_teams['A_S_TO']=0
games_teams['A_S_STL_A']=0
games_teams['A_S_BLK_A']=0
games_teams['A_S_TO_A']=0

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,fixPostSeasonAveragesGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,fixPostSeasonAveragesGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesPerGameGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesPerGameGeneric)

In [ ]:
games_teams['H_L5_STL']=0
games_teams['H_L5_BLK']=0
games_teams['H_L5_TO']=0
games_teams['H_L5_STL_A']=0
games_teams['H_L5_BLK_A']=0
games_teams['H_L5_TO_A']=0
games_teams['A_L5_STL']=0
games_teams['A_L5_BLK']=0
games_teams['A_L5_TO']=0
games_teams['A_L5_STL_A']=0
games_teams['A_L5_BLK_A']=0
games_teams['A_L5_TO_A']=0

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,fixPostSeasonAveragesGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,fixPostSeasonAveragesGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesPerGameGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesPerGameGenericL5)

In [ ]:
games_teams.to_csv('l5ptstoto.csv')

In [ ]:
def calculate_home_season_wins(row):
  wins=row.H_W
  games=row.H_G - 1
  if row.HOME_TEAM_WINS==1 :
    wins=wins-1
  final=0
  if games > 0 :
    final=wins/games
  return final

In [ ]:
def calculate_away_season_wins(row):
  wins=row.A_W
  games=row.A_G - 1
  if row.HOME_TEAM_WINS==0 :
    wins=wins-1
  final=0
  if games > 0 :
    final=wins/games
  return final

In [ ]:
def calculateTheWL5(table,year,team,max_games):
  results=table.loc[((table['HOME_TEAM_ID']==team) | (table['VISITOR_TEAM_ID']==team)) & (table['SEASON']==year)]
  year_classify=20000 + year  
  results=results.loc[(results['H_SEASON_ID']==year_classify)|(results['A_SEASON_ID']==year_classify)]
  
  games_played=2
  while games_played < 7 :
    game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
    previous_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-1)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-1)))]
    games_played=games_played+1 
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          table.loc[game_info.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] 
          results.loc[game_info.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] 
      else :
          table.loc[game_info.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0])
          results.loc[game_info.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) 
    else :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          table.loc[game_info.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] 
          results.loc[game_info.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] 
      
      else :
          table.loc[game_info.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0])
          results.loc[game_info.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) 
       
  while games_played < max_games :
    game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
    previous_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-1)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-1)))]
    previous6_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-6)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-6)))]
    games_played=games_played+1 
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
          else :
            table.loc[game_info.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
            results.loc[game_info.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
        else :
            table.loc[game_info.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0]) 
            results.loc[game_info.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
    else :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
          else :
            table.loc[game_info.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
            results.loc[game_info.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
        else :
            table.loc[game_info.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0]) 
            results.loc[game_info.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
  #
  game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
  previous_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-1)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-1)))]
  previous6_game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==(games_played-6)))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==(games_played-6)))]
  game_info1=game_info.loc[game_info['HOME_TEAM_ID']==team]
  game_info2=game_info.loc[game_info['VISITOR_TEAM_ID']==team]
  if (not(game_info1.empty)) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info1.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info1.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
          else :
            table.loc[game_info1.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
            results.loc[game_info1.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info1.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info1.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
        else :
            table.loc[game_info1.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0]) 
            results.loc[game_info1.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
  if (not(game_info2.empty)) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info2.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info2.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
          else :
            table.loc[game_info2.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
            results.loc[game_info2.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info2.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info2.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
        else :
            table.loc[game_info2.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0]) 
            results.loc[game_info2.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])

  return table 

In [ ]:
def fixPostSeasonAveragesWL5(table,year,team,max_games):
  results=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_G']==max_games)) | ((table['VISITOR_TEAM_ID']==team)&(table['A_G']==max_games))) & (table['SEASON']==year)]
  results_regular=table.loc[((table['HOME_TEAM_ID']==team) | (table['VISITOR_TEAM_ID']==team)) & (table['SEASON']==year)]
  last_regular_season_game_date=results['GAME_DATE_EST'].min()
  lrsg=results.loc[results['GAME_DATE_EST'].str.startswith(last_regular_season_game_date)]
  last=lrsg.copy()
  results2=results.copy()
  results=results.drop(index=lrsg.index)
  amount_of_games=len(results)
  index=calculateOrderBasedOnDate(results)
  games_back=5
  amount_fixed=0
  while amount_fixed < amount_of_games :
    game_info_temp=results.loc[index[amount_fixed]]
    game_info=results.loc[results['GAME_ID']==game_info_temp['GAME_ID']]
    if amount_fixed==0 :
      previous_game_info=last
    else :
      previous_game_info_temp=results.loc[index[amount_fixed-1]]
      previous_game_info=results.loc[results['GAME_ID']==previous_game_info_temp['GAME_ID']]
    if (5-amount_fixed)>0 :
      previous6_game_info=results_regular.loc[((results_regular['HOME_TEAM_ID']==team)&(results_regular['H_G']==(max_games-(5-amount_fixed))))|((results_regular['VISITOR_TEAM_ID']==team)&(results_regular['A_G']==(max_games-(5-amount_fixed))))]
    elif (5-amount_fixed)==0 :
      previous6_game_info=last
    else :
      previous6_game_info_temp=results.loc[index[amount_fixed-6]]
      previous6_game_info=results.loc[results['GAME_ID']==previous6_game_info_temp['GAME_ID']]
    amount_fixed=amount_fixed+1
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
          else :
            table.loc[game_info.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
            results.loc[game_info.index,['H_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
        else :
            table.loc[game_info.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0]) 
            results.loc[game_info.index,['H_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
    else :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - previous6_game_info['HOME_TEAM_WINS'].values[0]
          else :
            table.loc[game_info.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
            results.loc[game_info.index,['A_L5_W']]=previous_game_info['H_L5_W'].values[0] + previous_game_info['HOME_TEAM_WINS'].values[0] - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
      else :
        if(previous6_game_info['HOME_TEAM_ID'].values[0]==team) :
            table.loc[game_info.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
            results.loc[game_info.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - previous6_game_info['HOME_TEAM_WINS'].values[0]
        else :
            table.loc[game_info.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0]) 
            results.loc[game_info.index,['A_L5_W']]=previous_game_info['A_L5_W'].values[0] + (1 - previous_game_info['HOME_TEAM_WINS'].values[0]) - (1 - previous6_game_info['HOME_TEAM_WINS'].values[0])
  return table 

In [ ]:
def calculateTheSeasonAveragesPerGameWL5(table,year,team,max_games):
  results=table.loc[((table['HOME_TEAM_ID']==team) | (table['VISITOR_TEAM_ID']==team)) & (table['SEASON']==year)]
  year_classify=20000 + year  
  results=results.loc[(results['H_SEASON_ID']==year_classify)|(results['A_SEASON_ID']==year_classify)]
  games_played=2
  while games_played < 7 :
    game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
    previous_game_info=game_info
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
        table.loc[game_info.index,['H_L5_WP']]=game_info['H_L5_W'].values[0]/(games_played-1)       
    else :
      if(previous_game_info['VISITOR_TEAM_ID'].values[0]==team) :
        table.loc[game_info.index,['A_L5_WP']]=previous_game_info['A_L5_W'].values[0]/(games_played-1)
    games_played=games_played + 1
  while games_played < max_games :
    game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
    previous_game_info=game_info
    if (game_info['HOME_TEAM_ID'].values[0]==team) :
      if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
        table.loc[game_info.index,['H_L5_WP']]=game_info['H_L5_W'].values[0]/5     
    else :
      if(previous_game_info['VISITOR_TEAM_ID'].values[0]==team) :
        table.loc[game_info.index,['A_L5_WP']]=previous_game_info['A_L5_W'].values[0]/5
    games_played=games_played + 1
  game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_G']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_G']==games_played))]
  previous_game_info=game_info
  game_info1=game_info.loc[game_info['HOME_TEAM_ID']==team]
  game_info2=game_info.loc[game_info['VISITOR_TEAM_ID']==team]
  if (not(game_info1.empty)) :
      if(not(game_info1.empty)) :
        table.loc[game_info1.index,['H_L5_WP']]=game_info1['H_L5_W'].values[0]/5  
  if (not(game_info2.empty)) :
      if(not(game_info2.empty)) :
        table.loc[game_info2.index,['A_L5_WP']]=game_info2['A_L5_W'].values[0]/5
        
  return table  

In [ ]:
games_teams['H_S_WP']=games_teams.apply(lambda row: calculate_home_season_wins(row), axis=1)
games_teams['A_S_WP']=games_teams.apply(lambda row: calculate_away_season_wins(row), axis=1)

In [ ]:
games_teams['H_L5_W']=0
games_teams['A_L5_W']=0
games_teams['H_L5_WP']=0
games_teams['A_L5_WP']=0

In [ ]:
max_games=82
#65
ids_of_teams=[1610612737,1610612738,1610612740,1610612741,1610612742,1610612743,1610612745,
              1610612746,1610612747,1610612748,1610612749,1610612750,1610612751,1610612752,
              1610612753,1610612754,1610612755,1610612756,1610612757,1610612758,1610612759,
              1610612760,1610612761,1610612762,1610612763,1610612764,1610612765,1610612766,1610612739,1610612744]
number_of_2019_games=[67,72,72,65,75,73,72,72,71,73,73,64,72,66,73,73,73,73,74,72,71,72,72,72,73,72,66,65,65,65]
for k in range(27):
  for i in range(2003,2021,1) :
    if i==2011:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],66)
    elif i==2019:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],number_of_2019_games[k])
    elif i==2020:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],72)
    elif i==2021:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],68)
    elif i==2012 :
      if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
        games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],81)
      else :
        games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],max_games)
    else :
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],max_games)
for k in range(27,28,1):
  for i in range(2004,2021,1) :
    if i==2011:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],66)
    elif i==2019:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],number_of_2019_games[k])
    elif i==2020:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],72)
    elif i==2021:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],68)
    elif i==2012 :
      if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
        games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],81)
      else :
        games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],max_games)
    else :
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],max_games)
for k in range(28,30,1):
  for i in range(2003,2021,1) :
    if i==2011:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],66)
    elif i==2019:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],number_of_2019_games[k])
    elif i==2020:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],72)
    elif i==2021:
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],68)
    elif i==2012 :
      if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
        games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],81)
      else :
        games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],max_games)
    else :
      games_teams=calculateTheWL5(games_teams,i,ids_of_teams[k],max_games)

In [ ]:
for k in range(27):
  for i in range(2003,2021,1) :
    if i==2011:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],66)
    elif i==2019:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],number_of_2019_games[k])
    elif i==2020:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],72)
    elif i==2021:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],68)
    elif i==2012 :
      if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
        games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],81)
      else :
        games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],max_games)
    else :
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],max_games)

for k in range(27,28,1):
  for i in range(2004,2021,1) :
    if i==2011:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],66)
    elif i==2019:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],number_of_2019_games[k])
    elif i==2020:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],72)
    elif i==2021:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],68)
    elif i==2012 :
      if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
        games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],81)
      else :
        games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],max_games)
    else :
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],max_games)


for k in range(28,30,1):
  for i in range(2003,2021,1) :
    if i==2011:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],66)
    elif i==2019:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],number_of_2019_games[k])
    elif i==2020:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],72)
    elif i==2021:
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],68)
    elif i==2012 :
      if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
        games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],81)
      else :
        games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],max_games)
    else :
      games_teams=fixPostSeasonAveragesWL5(games_teams,i,ids_of_teams[k],max_games)



In [ ]:
for k in range(27):
  for i in range(2003,2021,1) :
    if i==2011:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],66)
    elif i==2019:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],number_of_2019_games[k])
    elif i==2020:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],72)
    elif i==2021:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],68)
    elif i==2012 :
      if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
        games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],81)
      else :
        games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],max_games)
    else :
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],max_games)

for k in range(27,28,1):
  for i in range(2004,2021,1) :
    if i==2011:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],66)
    elif i==2019:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],number_of_2019_games[k])
    elif i==2020:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],72)
    elif i==2021:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],68)
    elif i==2012 :
      if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
        games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],81)
      else :
        games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],max_games)
    else :
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],max_games)
for k in range(28,30,1):
  for i in range(2003,2021,1) :
    if i==2011:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],66)
    elif i==2019:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],number_of_2019_games[k])
    elif i==2020:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],72)
    elif i==2021:
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],68)
    elif i==2012 :
      if ((ids_of_teams[k]==1610612754) | (ids_of_teams[k]==1610612738)):
        games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],81)
      else :
        games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],max_games)
    else :
      games_teams=calculateTheSeasonAveragesPerGameWL5(games_teams,i,ids_of_teams[k],max_games)

In [ ]:
games_teams.to_csv('ptstotol5withwins.csv')

In [ ]:
games_teams=pd.read_csv('/content/ptstotol5withwins.csv')
games_teams.drop('Unnamed: 0', inplace=True, axis=1)
details=pd.read_csv('/content/games_details.csv',low_memory=False)

In [ ]:
def calculate_home_season_netrating(row):
  off_r=row.H_S_PTS
  def_r=row.H_S_PTS_A
  final=off_r - def_r
  return final

def calculate_away_season_netrating(row):
  off_r=row.A_S_PTS
  def_r=row.A_S_PTS_A
  final=off_r - def_r
  return final

In [ ]:
def calculate_home_l5_netrating(row):
  off_r=row.H_L5_PTS
  def_r=row.H_L5_PTS_A
  final=off_r - def_r
  return final

def calculate_away_l5_netrating(row):
  off_r=row.A_L5_PTS
  def_r=row.A_L5_PTS_A
  final=off_r - def_r
  return final

In [ ]:
games_teams['H_S_NET_RATING']=games_teams.apply(lambda row: calculate_home_season_netrating(row), axis=1)
games_teams['A_S_NET_RATING']=games_teams.apply(lambda row: calculate_away_season_netrating(row), axis=1)
games_teams['H_L5_NET_RATING']=games_teams.apply(lambda row: calculate_home_l5_netrating(row), axis=1)
games_teams['A_L5_NET_RATING']=games_teams.apply(lambda row: calculate_away_l5_netrating(row), axis=1)

In [ ]:
columns=[['FGA','FGA_home','FGA_away'],['FG3A','FG3A_home','FG3A_away'],['FTA','FTA_home','FTA_away'],['FGM','FGM_home','FGM_away'],['FG3M','FG3M_home','FG3M_away'],['FTM','FTM_home','FTM_away']]
for column in columns: 
  games_teams[column[1]]=0
  games_teams[column[2]]=0
for column in columns :
  games_teams[column[1]]=games_teams.apply(lambda row:addStatsToGames(row,details,'HOME_TEAM_ID',column[0]), axis=1)
for column in columns :
  games_teams[column[2]]=games_teams.apply(lambda row:addStatsToGames(row,details,'VISITOR_TEAM_ID',column[0]), axis=1)
print(games_teams)

      GAME_DATE_EST   GAME_ID GAME_STATUS_TEXT  HOME_TEAM_ID  VISITOR_TEAM_ID  \
0        2022-03-12  22101005            Final    1610612748       1610612750   
1        2022-03-12  22101006            Final    1610612741       1610612739   
2        2022-03-12  22101007            Final    1610612759       1610612754   
3        2022-03-12  22101008            Final    1610612744       1610612749   
4        2022-03-12  22101009            Final    1610612743       1610612761   
...             ...       ...              ...           ...              ...   
25762    2014-10-06  11400007            Final    1610612737       1610612740   
25763    2014-10-06  11400004            Final    1610612741       1610612764   
25764    2014-10-06  11400005            Final    1610612747       1610612743   
25765    2014-10-05  11400002            Final    1610612761       1610612758   
25766    2014-10-04  11400001            Final    1610612748       1610612740   

       SEASON  TEAM_ID_home

In [ ]:
columns=[['OREB','OREB_home','OREB_away'],['DREB','DREB_home','DREB_away'],['PF','PF_home','PF_away']]
for column in columns: 
  games_teams[column[1]]=0
  games_teams[column[2]]=0
for column in columns :
  games_teams[column[1]]=games_teams.apply(lambda row:addStatsToGames(row,details,'HOME_TEAM_ID',column[0]), axis=1)
for column in columns :
  games_teams[column[2]]=games_teams.apply(lambda row:addStatsToGames(row,details,'VISITOR_TEAM_ID',column[0]), axis=1)
print(games_teams)

      GAME_DATE_EST   GAME_ID GAME_STATUS_TEXT  HOME_TEAM_ID  VISITOR_TEAM_ID  \
0        2022-03-12  22101005            Final    1610612748       1610612750   
1        2022-03-12  22101006            Final    1610612741       1610612739   
2        2022-03-12  22101007            Final    1610612759       1610612754   
3        2022-03-12  22101008            Final    1610612744       1610612749   
4        2022-03-12  22101009            Final    1610612743       1610612761   
...             ...       ...              ...           ...              ...   
25762    2014-10-06  11400007            Final    1610612737       1610612740   
25763    2014-10-06  11400004            Final    1610612741       1610612764   
25764    2014-10-06  11400005            Final    1610612747       1610612743   
25765    2014-10-05  11400002            Final    1610612761       1610612758   
25766    2014-10-04  11400001            Final    1610612748       1610612740   

       SEASON  TEAM_ID_home

In [ ]:
games_teams.to_csv('Allboxscorestats.csv')

In [ ]:
columns=['FGA','FG3A','FTA','FGM','FG3M','FTM','OREB','DREB','PF']

In [ ]:
games_teams['H_S_FGA']=0
games_teams['A_S_FGA']=0
games_teams['H_S_FGA_A']=0
games_teams['A_S_FGA_A']=0
games_teams['H_S_FG3A']=0
games_teams['A_S_FG3A']=0
games_teams['H_S_FG3A_A']=0
games_teams['A_S_FG3A_A']=0
games_teams['H_S_FTA']=0
games_teams['A_S_FTA']=0
games_teams['H_S_FTA_A']=0
games_teams['A_S_FTA_A']=0
games_teams['H_S_FGM']=0
games_teams['A_S_FGM']=0
games_teams['H_S_FGM_A']=0
games_teams['A_S_FGM_A']=0
games_teams['H_S_FG3M']=0
games_teams['A_S_FG3M']=0
games_teams['H_S_FG3M_A']=0
games_teams['A_S_FG3M_A']=0
games_teams['H_S_FTM']=0
games_teams['A_S_FTM']=0
games_teams['H_S_FTM_A']=0
games_teams['A_S_FTM_A']=0
games_teams['H_S_OREB']=0
games_teams['A_S_OREB']=0
games_teams['H_S_OREB_A']=0
games_teams['A_S_OREB_A']=0
games_teams['H_S_DREB']=0
games_teams['A_S_DREB']=0
games_teams['H_S_DREB_A']=0
games_teams['A_S_DREB_A']=0
games_teams['H_S_PF']=0
games_teams['A_S_PF']=0
games_teams['H_S_PF_A']=0
games_teams['A_S_PF_A']=0

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,fixPostSeasonAveragesGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,fixPostSeasonAveragesGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesPerGameGeneric)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesPerGameGeneric)

In [ ]:
games_teams['H_L5_FGA']=0
games_teams['A_L5_FGA']=0
games_teams['H_L5_FGA_A']=0
games_teams['A_L5_FGA_A']=0
games_teams['H_L5_FG3A']=0
games_teams['A_L5_FG3A']=0
games_teams['H_L5_FG3A_A']=0
games_teams['A_L5_FG3A_A']=0
games_teams['H_L5_FTA']=0
games_teams['A_L5_FTA']=0
games_teams['H_L5_FTA_A']=0
games_teams['A_L5_FTA_A']=0
games_teams['H_L5_FGM']=0
games_teams['A_L5_FGM']=0
games_teams['H_L5_FGM_A']=0
games_teams['A_L5_FGM_A']=0
games_teams['H_L5_FG3M']=0
games_teams['A_L5_FG3M']=0
games_teams['H_L5_FG3M_A']=0
games_teams['A_L5_FG3M_A']=0
games_teams['H_L5_FTM']=0
games_teams['A_L5_FTM']=0
games_teams['H_L5_FTM_A']=0
games_teams['A_L5_FTM_A']=0
games_teams['H_L5_OREB']=0
games_teams['A_L5_OREB']=0
games_teams['H_L5_OREB_A']=0
games_teams['A_L5_OREB_A']=0
games_teams['H_L5_DREB']=0
games_teams['A_L5_DREB']=0
games_teams['H_L5_DREB_A']=0
games_teams['A_L5_DREB_A']=0
games_teams['H_L5_PF']=0
games_teams['A_L5_PF']=0
games_teams['H_L5_PF_A']=0
games_teams['A_L5_PF_A']=0

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,fixPostSeasonAveragesGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,fixPostSeasonAveragesGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitFirst(games_teams,columns,calculateTheSeasonAveragesPerGameGenericL5)

In [ ]:
games_teams=genericFunctionCallerSplitSecond(games_teams,columns,calculateTheSeasonAveragesPerGameGenericL5)

In [ ]:
games_teams.to_csv('allstatswithl5averages.csv')

# True Shooting, Effective field goal calculation

In [ ]:
def calculate_home_season_true_shooting(row):
  pts=row.H_S_PTS
  fga=row.H_S_FGA
  fta=row.H_S_FTA
  final=0
  if ( fga + fta ) > 0 : 
    final= pts/(2*(fga + (0.44*fta)))
  return final

def calculate_away_season_true_shooting(row):
  pts=row.A_S_PTS
  fga=row.A_S_FGA
  fta=row.A_S_FTA
  final=0
  if ( fga + fta ) > 0 : 
    final= pts/(2*(fga + (0.44*fta)))
  return final

In [ ]:
def calculate_home_l5_true_shooting(row):
  pts=row.H_L5_PTS
  fga=row.H_L5_FGA
  fta=row.H_L5_FTA
  final=0
  if ( fga + fta ) > 0 : 
    final= pts/(2*(fga + (0.44*fta)))
  return final

def calculate_away_l5_true_shooting(row):
  pts=row.A_L5_PTS
  fga=row.A_L5_FGA
  fta=row.A_L5_FTA
  final=0
  if ( fga + fta ) > 0 : 
    final= pts/(2*(fga + (0.44*fta)))
  return final

In [ ]:
def calculate_home_season_true_shooting_allowed(row):
  pts=row.H_S_PTS_A
  fga=row.H_S_FGA_A
  fta=row.H_S_FTA_A
  final=0
  if ( fga + fta ) > 0 : 
    final= pts/(2*(fga + (0.44*fta)))
  return final

def calculate_away_season_true_shooting_allowed(row):
  pts=row.A_S_PTS_A
  fga=row.A_S_FGA_A
  fta=row.A_S_FTA_A
  final=0
  if ( fga + fta ) > 0 : 
    final= pts/(2*(fga + (0.44*fta)))
  return final

In [ ]:
def calculate_home_l5_true_shooting_allowed(row):
  pts=row.H_L5_PTS_A
  fga=row.H_L5_FGA_A
  fta=row.H_L5_FTA_A
  final=0
  if ( fga + fta ) > 0 : 
    final= pts/(2*(fga + (0.44*fta)))
  return final

def calculate_away_l5_true_shooting_allowed(row):
  pts=row.A_L5_PTS_A
  fga=row.A_L5_FGA_A
  fta=row.A_L5_FTA_A
  final=0
  if ( fga + fta ) > 0 : 
    final= pts/(2*(fga + (0.44*fta)))
  return final

In [ ]:
def calculate_home_season_efg_shooting(row) :
  fgm=row.H_S_FGM
  fg3m=row.H_S_FG3M
  fga=row.H_S_FGA
  final=0
  if fga > 0 :
    final=(fgm + (fg3m * 0.5))/fga
  return final 

def calculate_away_season_efg_shooting(row) :
  fgm=row.A_S_FGM
  fg3m=row.A_S_FG3M
  fga=row.A_S_FGA
  final=0
  if fga > 0 :
    final=(fgm + (fg3m * 0.5))/fga
  return final 

In [ ]:
def calculate_home_l5_efg_shooting(row) :
  fgm=row.H_L5_FGM
  fg3m=row.H_L5_FG3M
  fga=row.H_L5_FGA
  final=0
  if fga > 0 :
    final=(fgm + (fg3m * 0.5))/fga
  return final

def calculate_away_l5_efg_shooting(row) :
  fgm=row.A_L5_FGM
  fg3m=row.A_L5_FG3M
  fga=row.A_L5_FGA
  final=0
  if fga > 0 :
    final=(fgm + (fg3m * 0.5))/fga
  return final

In [ ]:
def calculate_home_season_efg_shooting_allowed(row) :
  fgm=row.H_S_FGM_A
  fg3m=row.H_S_FG3M_A
  fga=row.H_S_FGA_A
  final=0
  if fga > 0 :
    final=(fgm + (fg3m * 0.5))/fga
  return final 

def calculate_away_season_efg_shooting_allowed(row) :
  fgm=row.A_S_FGM_A
  fg3m=row.A_S_FG3M_A
  fga=row.A_S_FGA_A
  final=0
  if fga > 0 :
    final=(fgm + (fg3m * 0.5))/fga
  return final 

In [ ]:
def calculate_home_l5_efg_shooting_allowed(row) :
  fgm=row.H_L5_FGM_A
  fg3m=row.H_L5_FG3M_A
  fga=row.H_L5_FGA_A
  final=0
  if fga > 0 :
    final=(fgm + (fg3m * 0.5))/fga
  return final

def calculate_away_l5_efg_shooting_allowed(row) :
  fgm=row.A_L5_FGM_A
  fg3m=row.A_L5_FG3M_A
  fga=row.A_L5_FGA_A
  final=0
  if fga > 0 :
    final=(fgm + (fg3m * 0.5))/fga
  return final

In [ ]:
games_teams['H_S_TS']=games_teams.apply(lambda row: calculate_home_season_true_shooting(row), axis=1)
games_teams['A_S_TS']=games_teams.apply(lambda row: calculate_away_season_true_shooting(row), axis=1)
games_teams['H_L5_TS']=games_teams.apply(lambda row: calculate_home_l5_true_shooting(row), axis=1)
games_teams['A_L5_TS']=games_teams.apply(lambda row: calculate_away_l5_true_shooting(row), axis=1)

In [ ]:
games_teams['H_S_TS_A']=games_teams.apply(lambda row: calculate_home_season_true_shooting_allowed(row), axis=1)
games_teams['A_S_TS_A']=games_teams.apply(lambda row: calculate_away_season_true_shooting_allowed(row), axis=1)
games_teams['H_L5_TS_A']=games_teams.apply(lambda row: calculate_home_l5_true_shooting_allowed(row), axis=1)
games_teams['A_L5_TS_A']=games_teams.apply(lambda row: calculate_away_l5_true_shooting_allowed(row), axis=1)

In [ ]:
games_teams['H_S_EFG']=games_teams.apply(lambda row: calculate_home_season_efg_shooting(row), axis=1)
games_teams['A_S_EFG']=games_teams.apply(lambda row: calculate_away_season_efg_shooting(row), axis=1)
games_teams['H_L5_EFG']=games_teams.apply(lambda row: calculate_home_l5_efg_shooting(row), axis=1)
games_teams['A_L5_EFG']=games_teams.apply(lambda row: calculate_away_l5_efg_shooting(row), axis=1)

In [ ]:
games_teams['H_S_EFG_A']=games_teams.apply(lambda row: calculate_home_season_efg_shooting_allowed(row), axis=1)
games_teams['A_S_EFG_A']=games_teams.apply(lambda row: calculate_away_season_efg_shooting_allowed(row), axis=1)
games_teams['H_L5_EFG_A']=games_teams.apply(lambda row: calculate_home_l5_efg_shooting_allowed(row), axis=1)
games_teams['A_L5_EFG_A']=games_teams.apply(lambda row: calculate_away_l5_efg_shooting_allowed(row), axis=1)

In [ ]:
games_teams.to_csv('allstatsaveragesplusshootingadvanced.csv')

# Four Factors 


In [3]:
games_teams=pd.read_csv('/content/allstatsaveragesplusshootingadvanced.csv')
games_teams.drop('Unnamed: 0', inplace=True, axis=1)

In [4]:
def calculate_PossesionsHome(row):
  total= 0.96*(row.H_S_FGA - row.H_S_OREB - row.H_S_TO + (0.475*(row.H_S_FTA)))
  return total

def calculate_PossesionsAway(row):
  total= 0.96*(row.A_S_FGA - row.A_S_OREB - row.A_S_TO + (0.475*(row.A_S_FTA)))
  return total

def calculate_PossesionsHomel5(row):
  total= 0.96*(row.H_L5_FGA - row.H_L5_OREB - row.H_L5_TO + (0.475*(row.H_L5_FTA)))
  return total

def calculate_PossesionsAwayl5(row):
  total= 0.96*(row.A_L5_FGA - row.A_L5_OREB - row.A_L5_TO + (0.475*(row.A_L5_FTA)))
  return total

def calculate_PossesionsHome_A(row):
  total= 0.96*(row.H_S_FGA_A - row.H_S_OREB_A - row.H_S_TO_A + (0.475*(row.H_S_FTA_A)))
  return total

def calculate_PossesionsAway_A(row):
  total= 0.96*(row.A_S_FGA_A - row.A_S_OREB_A - row.A_S_TO_A + (0.475*(row.A_S_FTA_A)))
  return total

def calculate_PossesionsHomel5_A(row):
  total= 0.96*(row.H_L5_FGA_A - row.H_L5_OREB_A - row.H_L5_TO_A + (0.475*(row.H_L5_FTA_A)))
  return total

def calculate_PossesionsAwayl5_A(row):
  total= 0.96*(row.A_L5_FGA_A - row.A_L5_OREB_A - row.A_L5_TO_A + (0.475*(row.A_L5_FTA_A)))
  return total


In [5]:
games_teams['H_S_P']=games_teams.apply(lambda row: calculate_PossesionsHome(row), axis=1)
games_teams['A_S_P']=games_teams.apply(lambda row: calculate_PossesionsAway(row), axis=1)
games_teams['H_L5_P']=games_teams.apply(lambda row: calculate_PossesionsHomel5(row), axis=1)
games_teams['A_L5_P']=games_teams.apply(lambda row: calculate_PossesionsAwayl5(row), axis=1)
games_teams['H_S_P_A']=games_teams.apply(lambda row: calculate_PossesionsHome_A(row), axis=1)
games_teams['A_S_P_A']=games_teams.apply(lambda row: calculate_PossesionsAway_A(row), axis=1)
games_teams['H_L5_P_A']=games_teams.apply(lambda row: calculate_PossesionsHomel5_A(row), axis=1)
games_teams['A_L5_P_A']=games_teams.apply(lambda row: calculate_PossesionsAwayl5_A(row), axis=1)

In [6]:
def calculate_oe_h(row):
  total=0
  if row.H_S_P>0 :
    total= (row.H_S_PTS*100)/row.H_S_P
  return total
def calculate_oe_a(row):
  total=0
  if row.A_S_P>0 :
    total= (row.A_S_PTS*100)/row.A_S_P
  return total

def calculate_oe_h_l5(row):
  total=0
  if row.H_L5_P>0 :
    total= (row.H_L5_PTS*100)/row.H_L5_P
  return total
def calculate_oe_a_l5(row):
  total=0
  if row.A_L5_P>0 :
    total= (row.A_L5_PTS*100)/row.A_L5_P
  return total


In [7]:
def calculate_de_h(row):
  total=0
  if row.H_S_P>0 :
    total= (row.H_S_PTS_A*100)/row.H_S_P
  return total
def calculate_de_a(row):
  total=0
  if row.A_S_P>0 :
    total= (row.A_S_PTS_A*100)/row.A_S_P
  return total

def calculate_de_h_l5(row):
  total=0
  if row.H_L5_P>0 :
    total= (row.H_L5_PTS_A*100)/row.H_L5_P
  return total
def calculate_de_a_l5(row):
  total=0
  if row.A_L5_P>0 :
    total= (row.A_L5_PTS_A*100)/row.A_L5_P
  return total


In [8]:
games_teams['H_S_OE']=games_teams.apply(lambda row: calculate_oe_h(row), axis=1)
games_teams['A_S_OE']=games_teams.apply(lambda row: calculate_oe_a(row), axis=1)
games_teams['H_L5_OE']=games_teams.apply(lambda row: calculate_oe_h_l5(row), axis=1)
games_teams['A_L5_OE']=games_teams.apply(lambda row: calculate_oe_a_l5(row), axis=1)
games_teams['H_S_DE']=games_teams.apply(lambda row: calculate_de_h(row), axis=1)
games_teams['A_S_DE']=games_teams.apply(lambda row: calculate_de_a(row), axis=1)
games_teams['H_L5_DE']=games_teams.apply(lambda row: calculate_de_h_l5(row), axis=1)
games_teams['A_L5_DE']=games_teams.apply(lambda row: calculate_de_a_l5(row), axis=1)

In [9]:
def calculate_tor_h(row):
  total=0
  if row.H_S_P>0 :
    total= row.H_S_TO/row.H_S_P
  return total
def calculate_tor_a(row):
  total=0
  if row.A_S_P>0 :
    total= row.A_S_TO/row.A_S_P
  return total

def calculate_tor_h_l5(row):
  total=0
  if row.H_L5_P>0 :
    total= row.H_L5_TO/row.H_L5_P
  return total
def calculate_tor_a_l5(row):
  total=0
  if row.A_L5_P>0 :
    total= row.A_L5_TO/row.A_L5_P
  return total


In [10]:
games_teams['H_S_TOR']=games_teams.apply(lambda row: calculate_tor_h(row), axis=1)
games_teams['A_S_TOR']=games_teams.apply(lambda row: calculate_tor_a(row), axis=1)
games_teams['H_L5_TOR']=games_teams.apply(lambda row: calculate_tor_h_l5(row), axis=1)
games_teams['A_L5_TOR']=games_teams.apply(lambda row: calculate_tor_a_l5(row), axis=1)

In [11]:
def calculate_oreb_pct_h(row):
  total=0
  base=row.H_S_OREB + row.A_S_DREB
  if base>0 :
    total= row.H_S_OREB/base
  return total
def calculate_oreb_pct_a(row):
  total=0
  base=row.A_S_OREB + row.H_S_DREB
  if base>0 :
    total= row.A_S_OREB/base
  return total

def calculate_oreb_pct_h_l5(row):
  total=0
  base=row.H_L5_OREB + row.A_L5_DREB
  if base>0 :
    total= row.H_L5_OREB/base
  return total
def calculate_oreb_pct_a_l5(row):
  total=0
  base=row.A_L5_OREB + row.H_L5_DREB
  if base>0 :
    total= row.A_L5_OREB/base
  return total



In [12]:
games_teams['H_S_OREB_PCT']=games_teams.apply(lambda row: calculate_oreb_pct_h(row), axis=1)
games_teams['A_S_OREB_PCT']=games_teams.apply(lambda row: calculate_oreb_pct_a(row), axis=1)
games_teams['H_L5_OREB_PCT']=games_teams.apply(lambda row: calculate_oreb_pct_h_l5(row), axis=1)
games_teams['A_L5_OREB_PCT']=games_teams.apply(lambda row: calculate_oreb_pct_a_l5(row), axis=1)

In [13]:
def calculate_ftr_h(row):
  total=0
  if row.H_S_FGA>0 :
    total= row.H_S_FTA/row.H_S_FGA
  return total
def calculate_ftr_a(row):
  total=0
  if row.A_S_FGA>0  :
    total= row.A_S_FTA/row.A_S_FGA
  return total

def calculate_ftr_h_l5(row):
  total=0
  if row.H_L5_FGA>0 :
    total= row.H_L5_FTA/row.H_L5_FGA
  return total
def calculate_ftr_a_l5(row):
  total=0
  if row.A_L5_FGA>0  :
    total= row.A_L5_FTA/row.A_L5_FGA
  return total


In [14]:
games_teams['H_S_FTR']=games_teams.apply(lambda row: calculate_ftr_h(row), axis=1)
games_teams['A_S_FTR']=games_teams.apply(lambda row: calculate_ftr_a(row), axis=1)
games_teams['H_L5_FTR']=games_teams.apply(lambda row: calculate_ftr_h_l5(row), axis=1)
games_teams['A_L5_FTR']=games_teams.apply(lambda row: calculate_ftr_a_l5(row), axis=1)

In [15]:
games_teams.to_csv('allstatplusfourfactors.csv')

In [16]:
def calculate_home_season_winslosses(row):
  wins=row.H_W
  losses=row.H_L
  if row.HOME_TEAM_WINS==1 :
    wins=wins-1
  else :
    losses=losses - 1
  final=0
  final=wins - losses
  return final

In [17]:
def calculate_away_season_winslosses(row):
  wins=row.A_W
  losses=row.H_L 
  if row.HOME_TEAM_WINS==0 :
    wins=wins-1
  else :
    losses=losses - 1
  final=0
  final=wins - losses
  return final

In [18]:
games_teams['H_S_WL']=games_teams.apply(lambda row: calculate_home_season_winslosses(row), axis=1)
games_teams['A_S_WL']=games_teams.apply(lambda row: calculate_away_season_winslosses(row), axis=1)

# Total Games Head 2 Head Data Plus Elo

In [ ]:
data_teams=pd.read_csv('/content/allstatsplusfourfactors.csv')
data_teams.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
games_teams=data_teams.copy()

In [19]:
def caclulateTotalGamesHome(row) :
  return row.H_G
def caclulateTotalGamesAway(row) :
  return row.A_G

In [20]:
games_teams['H_TG']=games_teams.apply(lambda row: caclulateTotalGamesHome(row), axis=1)
games_teams['A_TG']=games_teams.apply(lambda row: caclulateTotalGamesAway(row), axis=1)

In [21]:
def calculateTotalGames(table,year,team,max_games,columns):
  results=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_G']==max_games)) | ((table['VISITOR_TEAM_ID']==team)&(table['A_G']==max_games))) & (table['SEASON']==year)]
  results_regular=table.loc[((table['HOME_TEAM_ID']==team) | (table['VISITOR_TEAM_ID']==team)) & (table['SEASON']==year)]
  last_regular_season_game_date=results['GAME_DATE_EST'].min()
  lrsg=results.loc[results['GAME_DATE_EST'].str.startswith(last_regular_season_game_date)]
  last=lrsg.copy()
  results2=results.copy()
  results=results.drop(index=lrsg.index)
  amount_of_games=len(results)
  index=calculateOrderBasedOnDate(results)
  amount_fixed=0
  while amount_fixed < amount_of_games :
    game_info_temp=results.loc[index[amount_fixed]]
    game_info=results.loc[results['GAME_ID']==game_info_temp['GAME_ID']]
    if(game_info['HOME_TEAM_ID'].values[0]==team) :
      table.loc[game_info.index,['H_TG']]= max_games + 1 + amount_fixed
    else :
      table.loc[game_info.index,['A_TG']]= max_games + 1 + amount_fixed
    amount_fixed=amount_fixed + 1
  return table  

In [27]:
games_teams=genericFunctionCaller(games_teams,['null'],calculateTotalGames)

In [28]:
def calculateHeadToHeadMatchups(table,year,team,max_games,columns):
  results=table.loc[((table['HOME_TEAM_ID']==team) | (table['VISITOR_TEAM_ID']==team)) & (table['SEASON']==year)]
  year_classify=20000 + year  
  results=results.loc[(results['H_SEASON_ID']==year_classify)|(results['A_SEASON_ID']==year_classify)]
  games_played=2
  results_playoff_only=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_G']==max_games)) | ((table['VISITOR_TEAM_ID']==team)&(table['A_G']==max_games))) & (table['SEASON']==year)]
  amount_of_games_added=len(results_playoff_only) - 1
  max_all=max_games + amount_of_games_added + 1
  while games_played< max_all :
    game_info=results.loc[((results['HOME_TEAM_ID']==team)&(results['H_TG']==games_played))|((results['VISITOR_TEAM_ID']==team)&(results['A_TG']==games_played))]
    other_team_id=0
    if(game_info['HOME_TEAM_ID'].values[0]==team) :
      other_team_id=game_info['VISITOR_TEAM_ID'].values[0]
    else :
      other_team_id=game_info['HOME_TEAM_ID'].values[0]
    head_to_head=results.loc[(((results['HOME_TEAM_ID']==team)&(results['H_TG']<games_played)) & (results['VISITOR_TEAM_ID']==other_team_id))|(((results['VISITOR_TEAM_ID']==team)&(results['A_TG']<games_played))& (results['HOME_TEAM_ID']==other_team_id))]
    amount_of_games=len(head_to_head)
    index=calculateOrderBasedOnDate(head_to_head)
    amount_fixed=0
    winper=0
    while amount_fixed<amount_of_games :
      game_temp=head_to_head.loc[index[amount_fixed]]
      game=head_to_head.loc[head_to_head['GAME_ID']==game_temp['GAME_ID']]
      if (game['HOME_TEAM_ID'].values[0]==team) :
        winper=winper + game['HOME_TEAM_WINS'].values[0]
      else :
        winper=winper + (1 - game['HOME_TEAM_WINS'].values[0])
      amount_fixed=amount_fixed + 1
    if amount_of_games > 0 :
      winper=winper/amount_of_games
    if(game_info['HOME_TEAM_ID'].values[0]==team) :
      table.loc[game_info.index,'H_S_H2H']=winper
      results.loc[game_info.index,'H_S_H2H']=winper
    else :
      table.loc[game_info.index,'A_S_H2H']=winper
      results.loc[game_info.index,'A_S_H2H']=winper    
    games_played=games_played + 1
  return table

In [29]:
games_teams['H_S_H2H']=0
games_teams['A_S_H2H']=0

In [30]:
games_teams=genericFunctionCaller(games_teams,['null'],calculateHeadToHeadMatchups)

In [31]:
games_teams.to_csv('allplushead2head.csv')

In [32]:
def calculateEloTeamsTill2018(table,year,max_games) :
  h_y=20000 + year
  team_ids=[1610612737,1610612738,1610612740,1610612741,1610612742,1610612743,1610612745,
                1610612746,1610612747,1610612748,1610612749,1610612750,1610612751,1610612752,
                1610612753,1610612754,1610612755,1610612756,1610612757,1610612758,1610612759,
                1610612760,1610612761,1610612762,1610612763,1610612764,1610612765,1610612766,1610612739,1610612744]
  teams_ids=team_ids
  number_of_2019_games=[67,72,72,65,75,73,72,72,71,73,73,64,72,66,73,73,73,73,74,72,71,72,72,72,73,72,66,65,65,65]
  amount_of_teams=len(team_ids)
  games_played=2
  if year>2003 :
    for team in team_ids :
      game_info=table.loc[((table['HOME_TEAM_ID']==team)&(table['H_G']==1))|((table['VISITOR_TEAM_ID']==team)&(table['A_G']==1))]
      size=len(game_info)
      if size > 0 :
        results1=table.loc[((table['HOME_TEAM_ID']==team) | (table['VISITOR_TEAM_ID']==team)) & (table['SEASON']==(year-1))]
        last_season_game_date=results1['GAME_DATE_EST'].max()
        previous_game_info=results1.loc[results1['GAME_DATE_EST'].str.startswith(last_season_game_date)]
        size2=len(previous_game_info)
        if(size2>0) :
          if (game_info['HOME_TEAM_ID'].values[0]==team) :
            if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
              table.loc[game_info.index,['H_S_ELO']]=int((previous_game_info['H_S_ELO'].values[0]*0.75) + (0.25*1505))
            else : 
              table.loc[game_info.index,['H_S_ELO']]=int((previous_game_info['A_S_ELO'].values[0]*0.75) + (0.25*1505))
          else :
            if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
              table.loc[game_info.index,['A_S_ELO']]=int((previous_game_info['H_S_ELO'].values[0]*0.75) + (0.25*1505))
            else : 
              table.loc[game_info.index,['A_S_ELO']]=int((previous_game_info['A_S_ELO'].values[0]*0.75) + (0.25*1505))
  while games_played<max_games :
    for singular_index in range(amount_of_teams) :
      team=teams_ids[singular_index]
      if ((year==2012) & ((team==1610612754) | (team==1610612738))) :
        if games_played < 81 :
          game_info=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_G']==games_played))|((table['VISITOR_TEAM_ID']==team)&(table['A_G']==games_played)))&(table['H_SEASON_ID']==h_y)]
          size=len(game_info)
        else :
          size=0
      else :
        game_info=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_G']==games_played))|((table['VISITOR_TEAM_ID']==team)&(table['A_G']==games_played)))&(table['H_SEASON_ID']==h_y)]
        size=len(game_info)
      if size > 0 :
        previous_game_info=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_G']==(games_played-1)))|((table['VISITOR_TEAM_ID']==team)&(table['A_G']==(games_played-1))))&(table['H_SEASON_ID']==h_y)]
        if (game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
             elo_diff=(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])/400
             expected=1/(1+(10**(elo_diff)))
             if(previous_game_info['HOME_TEAM_WINS'].values[0]==1) :
               margin=previous_game_info['PTS_home'].values[0] - previous_game_info['PTS_away'].values[0]
               elo_diff_winner=abs(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])
             else :
               margin=previous_game_info['PTS_away'].values[0] - previous_game_info['PTS_home'].values[0]
               elo_diff_winner=abs(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])
             moving= 20*(((margin+3)**(0.8))/(7.5 + (0.006*elo_diff_winner)))
             table.loc[game_info.index,['H_S_ELO']]=int(previous_game_info['H_S_ELO'].values[0] + (previous_game_info['HOME_TEAM_WINS'].values[0] - expected)*moving)
            # results.loc[game_info.index,['H_S_ELO']]=previous_game_info['H_S_ELO'] + (previous_game_info['HOME_TEAM_WINS'] + expected)*moving
          else : 
             elo_diff=(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])/400
             expected=1/(1+(10**(elo_diff)))
             if(previous_game_info['HOME_TEAM_WINS'].values[0]==1) :
               margin=previous_game_info['PTS_home'].values[0] - previous_game_info['PTS_away'].values[0]
               elo_diff_winner=abs(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])
             else :
               margin=previous_game_info['PTS_away'].values[0] - previous_game_info['PTS_home'].values[0]
               elo_diff_winner=abs(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])
             moving= 20*(((margin+3)**(0.8))/(7.5 + (0.006*elo_diff_winner)))
             table.loc[game_info.index,['H_S_ELO']]=int(previous_game_info['A_S_ELO'].values[0] + ((1-previous_game_info['HOME_TEAM_WINS'].values[0]) - expected)*moving)
             #results.loc[game_info.index,['H_S_ELO']]=previous_game_info['A_S_ELO'] + 63#((1-previous_game_info['HOME_TEAM_WINS']) + expected)*moving
        else :
          if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
             elo_diff=(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])/400
             expected=1/(1+(10**(elo_diff)))
             if(previous_game_info['HOME_TEAM_WINS'].values[0]==1) :
               margin=previous_game_info['PTS_home'].values[0] - previous_game_info['PTS_away'].values[0]
               elo_diff_winner=abs(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])
             else :
               margin=previous_game_info['PTS_away'].values[0] - previous_game_info['PTS_home'].values[0]
               elo_diff_winner=abs(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])
             moving= 20*(((margin+3)**(0.8))/(7.5 + (0.006*elo_diff_winner)))
             table.loc[game_info.index,['A_S_ELO']]=int(previous_game_info['H_S_ELO'].values[0] + (previous_game_info['HOME_TEAM_WINS'].values[0] - expected)*moving)
            # results.loc[game_info.index,['A_S_ELO']]=previous_game_info['H_S_ELO'] + 63#(previous_game_info['HOME_TEAM_WINS'] + expected)*moving
          else : 
             elo_diff=(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])/400
             expected=1/(1+(10**(elo_diff)))
             if(previous_game_info['HOME_TEAM_WINS'].values[0]==1) :
               margin=previous_game_info['PTS_home'].values[0] - previous_game_info['PTS_away'].values[0]
               elo_diff_winner=abs(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])
             else :
               margin=previous_game_info['PTS_away'].values[0] - previous_game_info['PTS_home'].values[0]
               elo_diff_winner=abs(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])
             moving= 20*(((margin+3)**(0.8))/(7.5 + (0.006*elo_diff_winner)))
             table.loc[game_info.index,['A_S_ELO']]=int(previous_game_info['A_S_ELO'].values[0] + ((1-previous_game_info['HOME_TEAM_WINS'].values[0]) - expected)*moving)
            # results.loc[game_info.index,['A_S_ELO']]=previous_game_info['A_S_ELO'] + 63#((1-previous_game_info['HOME_TEAM_WINS']) + expected)*moving
    games_played=1+ games_played
  max_total_home=table['H_TG'].max()
  max_total_away=table['A_TG'].max()
  if max_total_home> max_total_away :
    max_total=max_total_home
  else:
    max_total=max_total_away
  while games_played<max_total :
    for singular_index in range(amount_of_teams) :
      team=teams_ids[singular_index]
      if ((year==2012) & ((team==1610612754) | (team==1610612738))) :
        game_info=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_TG']==(games_played-1)))|((table['VISITOR_TEAM_ID']==team)&(table['A_TG']==(games_played-1))))&(table['H_SEASON_ID']==h_y)]
        size=len(game_info)
      else :
        game_info=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_TG']==games_played))|((table['VISITOR_TEAM_ID']==team)&(table['A_TG']==games_played)))&(table['H_SEASON_ID']==h_y)]
        size=len(game_info)
      if size > 0 :
        if ((year==2012) & ((team==1610612754) | (team==1610612738))) :
          previous_game_info=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_TG']==(games_played-2)))|((table['VISITOR_TEAM_ID']==team)&(table['A_TG']==(games_played-2))))&(table['H_SEASON_ID']==h_y)]
        else :
          previous_game_info=table.loc[(((table['HOME_TEAM_ID']==team)&(table['H_TG']==(games_played-1)))|((table['VISITOR_TEAM_ID']==team)&(table['A_TG']==(games_played-1))))&(table['H_SEASON_ID']==h_y)]
        if (game_info['HOME_TEAM_ID'].values[0]==team) :
          if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
             elo_diff=(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])/400
             expected=1/(1+(10**(elo_diff)))
             if(previous_game_info['HOME_TEAM_WINS'].values[0]==1) :
               margin=previous_game_info['PTS_home'].values[0] - previous_game_info['PTS_away'].values[0]
               elo_diff_winner=abs(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])
             else :
               margin=previous_game_info['PTS_away'].values[0] - previous_game_info['PTS_home'].values[0]
               elo_diff_winner=abs(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])
             moving= 20*(((margin+3)**(0.8))/(7.5 + (0.006*elo_diff_winner)))
             table.loc[game_info.index,['H_S_ELO']]=int(previous_game_info['H_S_ELO'].values[0] + (previous_game_info['HOME_TEAM_WINS'].values[0] - expected)*moving)
            # results.loc[game_info.index,['H_S_ELO']]=previous_game_info['H_S_ELO'] + (previous_game_info['HOME_TEAM_WINS'] + expected)*moving
          else : 
             elo_diff=(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])/400
             expected=1/(1+(10**(elo_diff)))
             if(previous_game_info['HOME_TEAM_WINS'].values[0]==1) :
               margin=previous_game_info['PTS_home'].values[0] - previous_game_info['PTS_away'].values[0]
               elo_diff_winner=abs(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])
             else :
               margin=previous_game_info['PTS_away'].values[0] - previous_game_info['PTS_home'].values[0]
               elo_diff_winner=abs(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])
             moving= 20*(((margin+3)**(0.8))/(7.5 + (0.006*elo_diff_winner)))
             table.loc[game_info.index,['H_S_ELO']]=int(previous_game_info['A_S_ELO'].values[0] + ((1-previous_game_info['HOME_TEAM_WINS'].values[0]) - expected)*moving)
             #results.loc[game_info.index,['H_S_ELO']]=previous_game_info['A_S_ELO'] + 63#((1-previous_game_info['HOME_TEAM_WINS']) + expected)*moving
        else :
          if(previous_game_info['HOME_TEAM_ID'].values[0]==team) :
             elo_diff=(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])/400
             expected=1/(1+(10**(elo_diff)))
             if(previous_game_info['HOME_TEAM_WINS'].values[0]==1) :
               margin=previous_game_info['PTS_home'].values[0] - previous_game_info['PTS_away'].values[0]
               elo_diff_winner=abs(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])
             else :
               margin=previous_game_info['PTS_away'].values[0] - previous_game_info['PTS_home'].values[0]
               elo_diff_winner=abs(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])
             moving= 20*(((margin+3)**(0.8))/(7.5 + (0.006*elo_diff_winner)))
             table.loc[game_info.index,['A_S_ELO']]=int(previous_game_info['H_S_ELO'].values[0] + (previous_game_info['HOME_TEAM_WINS'].values[0] - expected)*moving)
            # results.loc[game_info.index,['A_S_ELO']]=previous_game_info['H_S_ELO'] + 63#(previous_game_info['HOME_TEAM_WINS'] + expected)*moving
          else : 
             elo_diff=(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])/400
             expected=1/(1+(10**(elo_diff)))
             if(previous_game_info['HOME_TEAM_WINS'].values[0]==1) :
               margin=previous_game_info['PTS_home'].values[0] - previous_game_info['PTS_away'].values[0]
               elo_diff_winner=abs(previous_game_info['H_S_ELO'].values[0] - previous_game_info['A_S_ELO'].values[0])
             else :
               margin=previous_game_info['PTS_away'].values[0] - previous_game_info['PTS_home'].values[0]
               elo_diff_winner=abs(previous_game_info['A_S_ELO'].values[0] - previous_game_info['H_S_ELO'].values[0])
             moving= 20*(((margin+3)**(0.8))/(7.5 + (0.006*elo_diff_winner)))
             table.loc[game_info.index,['A_S_ELO']]=int(previous_game_info['A_S_ELO'].values[0] + ((1-previous_game_info['HOME_TEAM_WINS'].values[0]) - expected)*moving)
    games_played=1+ games_played
  return table

In [33]:
def eloFunctionCall(games_teams,function_called): 
    for i in range(2003,2019,1) :
      if i==2011:
        games_teams=function_called(games_teams,i,66)
      else :
        games_teams=function_called(games_teams,i,82)
    return games_teams


In [34]:
games_teams['H_S_ELO']=1500
games_teams['A_S_ELO']=1500

In [35]:
games_teams=eloFunctionCall(games_teams,calculateEloTeamsTill2018) 

In [36]:
games_teams.to_csv('allstatsPlusElo.csv')

# Preparing Data

In [ ]:
data_teams=pd.read_csv('/content/allstatsPlusElo.csv')
data_teams.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
games_teams=data_teams.copy()

In [37]:
game_data_temp=games_teams.copy()
preseason=game_data_temp.loc[(game_data_temp['H_SEASON_ID']<20000)|(game_data_temp['A_SEASON_ID']<20000)]
game_data_temp=game_data_temp.drop(index=preseason.index)
unfinished_season=game_data_temp.loc[game_data_temp['SEASON']==2021]
game_data_temp=game_data_temp.drop(index=unfinished_season.index)
first_games=game_data_temp.loc[(game_data_temp['H_G']==1)|(game_data_temp['A_G']==1)]
game_data_temp=game_data_temp.drop(index=first_games.index)
game_data_temp.to_csv('game_data_teams_regular_season_and_playoffs_2003_2020.csv')